# 2. Properties of CO$_2$

## 2.1. Pure CO$_2$ enthalpy and Joule-Thomson coefficient

In [ ]:
import numpy as np
import xarray as xr
from dartsflash.dartsflash import DARTSFlash, CompData, R
from dartsflash.libflash import EoS, CubicEoS


# Define mixture and initialize Flash object
components = ["CO2"]
comp_data = CompData(components, setprops=True)
f = DARTSFlash(comp_data=comp_data)

# Add CubicEoS object
f.add_eos("CEOS", CubicEoS(comp_data, CubicEoS.PR), root_order=[EoS.MAX, EoS.MIN])
ceos = f.eos["CEOS"]

# Plot enthalpy diagram
state_spec = {"temperature": np.arange(263, 423, 1),
              "pressure": np.arange(1, 200, 1),
             }
compositions = {"CO2": 1.}
properties = {"H": ceos.H,
              "JT": ceos.JT,
             }

# Initialize PT-flash and evaluate PT-space
f.init_flash(flash_type=DARTSFlash.FlashType.PTFlash, eos_order=["CEOS"])

# Evaluate pure ph
props_1p = f.evaluate_phase_properties_1p(state_spec=state_spec, compositions=compositions,
                                          properties_to_evaluate=properties, mole_fractions=True)

# Evaluate flash and total enthalpy in PT-space
pt_flash_results = f.evaluate_flash(state_spec=state_spec, compositions=compositions, mole_fractions=True)
props_tot = f.evaluate_properties_np(state_spec=state_spec, compositions=compositions,
                                     state_variables=f.state_vars + components, flash_results=pt_flash_results,
                                     total_properties_to_evaluate={"H": EoS.Property.ENTHALPY})


## 2.1a. Enthalpy and isenthalpic cooling

In [ ]:
# Plot PT diagram of CO2 enthalpy
from dartsflash.plot import PlotFlash
pt_plot = PlotFlash.pt(f, pt_flash_results, composition=[1.], plot_phase_fractions=False, logP=False,
                       pt_props=props_tot, critical_point={"pressure": comp_data.Pc[0], "temperature": comp_data.Tc[0]})


## 2.1b. Joule-Thomson coefficient

In [ ]:
# Get reference data from NIST
from ref_data import get_nist_data

temperatures = np.array([0., 30., 60, 90]) + 273.15
ref_p, ref_JT = [[] for i, temp in enumerate(temperatures)], [[] for i, temp in enumerate(temperatures)]
for i, t in enumerate(temperatures):
    nist_data = get_nist_data(component="CO2", temperature=t, 
                              pressure=np.arange(25, 201, 25), include_phase_boundary=True)

    ref_p[i] = nist_data["pressure"]
    ref_JT[i] = nist_data["jt"]
    

In [ ]:
# Joule-Thomson coefficient CO2
from dartsflash.plot import PlotEoS
jt = PlotEoS.plot(f, props_1p, composition=[1.], x_var="pressure", prop_names=["JT"],
                  state={"temperature": temperatures}, datalabels=["T = {:.0f} K".format(t) for t in temperatures])

jt.draw_point(X=ref_p, Y=ref_JT)
jt.add_attributes(ax_labels=["pressure, bar", "JT, K/bar"], legend=True)


## 2.2. PH-diagram of CO$_2$

In [ ]:
#  PH diagram
from dartsflash.mixtures import VL
f = VL(comp_data=comp_data)

f.set_vl_eos("PR")
f.init_flash(flash_type=DARTSFlash.FlashType.PHFlash,
             t_min=100., t_max=1000., t_init=300., f_tol=1e-6, t_tol=1e-8)  # tolerance of enthalpy specification H-Hspec, and temperature bracketing in case of phase transition temperature

# Evaluate PH-flash
Prange = [10., 100.]
Trange = [260., 350.]
z = [1.]
Hrange = f.get_ranges(prange=Prange, trange=Trange, composition=z)
state_spec = {"pressure": np.linspace(Prange[0], Prange[1], 100),
              "enthalpy": np.linspace(Hrange[0], Hrange[1], 100)
             }

ph_flash_results = f.evaluate_flash_1c(state_spec=state_spec, print_state="PH flash")

# Plot PH diagram
from dartsflash.plot import PlotFlash
ph = PlotFlash.ph(f, ph_flash_results, composition=z, min_temp=250., max_temp=350., min_val=0., max_val=1.,
                  plot_phase_fractions=True, pt_props=None)


## 2.3. Isenthalpic cooling

In [ ]:
# Plot PT diagram of CO2 enthalpy
from dartsflash.plot import PlotFlash
pt_plot = PlotFlash.pt(f, pt_flash_results, composition=[1.], plot_phase_fractions=False, logP=False,
                       pt_props=props_tot, critical_point={"pressure": comp_data.Pc[0], "temperature": comp_data.Tc[0]})

R = 8.3144
p = 100.
enthalpies = [ceos.H(p, T=temp, n=[1.]) * R for temp in [ 333, 303, 273]]

pressures = [ph_flash_results.pressure.values for h in enthalpies]
temps = [ph_flash_results.sel(enthalpy=h, method='nearest').temp.values for h in enthalpies]

# Draw isenthalpic pressure drops
pt_plot.draw_line(temps, pressures)
